In [1]:
import os 
from dotenv import load_dotenv 
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq 
model = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002C8962BBEB0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002C8962F5360>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([
    HumanMessage(content="Hi, I'm a software engineer")
])

AIMessage(content="That's awesome! 👋\n\nWhat kind of software engineering do you do?  Web development, mobile apps, game development, something else entirely?  \n\nI'm always interested in learning more about what people do in this field.  Maybe you can tell me about a project you're working on or something you're excited about in the world of software engineering right now. 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 17, 'total_tokens': 99, 'completion_time': 0.149090909, 'prompt_time': 0.002002926, 'queue_time': 0.243375954, 'total_time': 0.151093835}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--84517848-a492-401e-a5dc-ae55e2c75ceb-0', usage_metadata={'input_tokens': 17, 'output_tokens': 82, 'total_tokens': 99})

In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory 
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory 

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store: 
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [6]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

config = {"configurable": {"session_id": "chatbot1"}}


In [13]:
with_message_history.invoke(
    [HumanMessage(content="Hi, I am John Doe. I work as a software engineer")],
    config=config
)

AIMessage(content="Hi John Doe! It's nice to finally know your name and profession.  Software engineering is a fascinating field. What kind of software do you specialize in? 💻  \n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 249, 'total_tokens': 289, 'completion_time': 0.072727273, 'prompt_time': 0.011849409, 'queue_time': 0.24484713, 'total_time': 0.084576682}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7ff97c61-e0f3-4591-a2d9-10868aed0686-0', usage_metadata={'input_tokens': 249, 'output_tokens': 40, 'total_tokens': 289})

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my profesion?")],
    config=config
)
response.content

"You said your profession is software engineering, John Doe. \n\nI remember what you tell me!  \n\nIs there anything else you'd like to chat about?  \n"

In [9]:
config = {"configurable": {"session_id": "chatbot2"}}
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

AIMessage(content="As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 14, 'total_tokens': 57, 'completion_time': 0.078181818, 'prompt_time': 0.001913477, 'queue_time': 0.244024152, 'total_time': 0.080095295}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--810bcdae-40f8-4185-bf0d-0aa83498642b-0', usage_metadata={'input_tokens': 14, 'output_tokens': 43, 'total_tokens': 57})

### Chat prompts

In [3]:
from langchain_groq import ChatGroq 
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder 

model = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model
chain

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [6]:
from langchain_core.messages import HumanMessage
chain.invoke({
    "messages": [HumanMessage(content="Hi, I am a software tester")]
})
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [7]:
config = {"configurable": {"session_id": "chatbot3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi, I am a Solution Architect")],
    config = config
)
response

AIMessage(content="Hello! That's great. I can be quite helpful for a Solution Architect. \n\nHow can I assist you today? \n\nDo you need help with:\n\n* **Brainstorming solutions for a particular problem?**\n* **Comparing different technologies or architectures?**\n* **Documenting a solution design?**\n* **Understanding complex technical concepts?**\n* **Generating code snippets or scripts?**\n\nJust let me know what's on your mind and I'll do my best to help!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 22, 'total_tokens': 132, 'completion_time': 0.2, 'prompt_time': 0.002179099, 'queue_time': 0.24803179100000003, 'total_time': 0.202179099}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--0dfed5f3-032d-45d2-ab86-a2dffa83cf44-0', usage_metadata={'input_tokens': 22, 'output_tokens': 110, 'total_tokens': 132})

In [12]:
# Add more complexity 
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer in {language}"),
        MessagesPlaceholder(variable_name="msg")
    ]
)
chain = prompt | model 

resp = chain.invoke({
    "msg": [HumanMessage(content="Hi, I am a Sales executive")],
    "language": "Hindi"
})
resp.content



'नमस्ते!  मी एक सहायक बनून येथे आहे. \n\nतुम्ही विक्री अधिकारी आहात हे मला माहित आहे. मला काय मदत करावी लागेल? \n\n'

In [14]:
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="msg"
)
config = {"configurable": {"session_id": "chatbot-test"}}
resp = with_message_history.invoke(
    {
        "msg": [HumanMessage(content="Hi, I am a Sales executive from Delhi")],
        "language":"Hindi"
    },
    config=config
)
resp.content

'नमस्ते! दिल्ली से एक सेल्स एक्जीक्यूटिव होने के नाते आपका स्वागत है!  \n\nआप मुझे कैसे मदद कर सकते हैं? \n\n'

In [16]:
response = with_message_history.invoke(
    {
        "msg": [HumanMessage(content="What is my profession?")],
        "language":"Hindi"
    },
    config=config
)
response.content

'आपने खुद ही बताया था कि आप एक सेल्स एक्जीक्यूटिव हैं।  \n'

### Managing Chat conversation history

In [22]:
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.messages.ai import AIMessage

trimmer = trim_messages(
    max_tokens = 70,
    strategy="last",
    token_counter=model,
    include_system=True,
    # allow_partials=False,
    start_on="human"
)
message = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi, I am Bob"),
    AIMessage(content="Hi Bob"),
    HumanMessage(content="What is 5 + 2"),
    AIMessage(content="7"),
    HumanMessage(content="What is the capital of France?"),
    AIMessage(content="Paris"),
    HumanMessage(content="How's the weather?"),
    AIMessage(content="Hot and humid"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No problem"),
]
trimmer.invoke(message)

d:\Code\Python\langchain-app\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Code\Python\langchain-app\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\indra\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this a

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, I am Bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi Bob', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 5 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='7', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Paris', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="How's the weather?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hot and humid', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem', additional_kwargs={}, response_metadata={})]

In [25]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough 

chain = (
    RunnablePassthrough.assign(messages = itemgetter("msg") | trimmer)
    | prompt 
    | model
)
chain.invoke({
    "msg": message + [HumanMessage(content="What answer did I get about the weather?")],
    "language": "Hindi"
})

AIMessage(content='आपको मैंने बताया था की मौसम गर्म और आर्द्र है। \n\n\n\nLet me know if you have any other questions!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 119, 'total_tokens': 151, 'completion_time': 0.058181818, 'prompt_time': 0.005351817, 'queue_time': 0.255038202, 'total_time': 0.063533635}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--ff086762-418e-4373-a4c6-44518d02bb5c-0', usage_metadata={'input_tokens': 119, 'output_tokens': 32, 'total_tokens': 151})

In [27]:
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="msg"
)
config = {"configurable": {"session_id": "chatbot-history"}}

response = with_message_history.invoke(
    {
        "msg": message + [HumanMessage(content="What did I ask about the weather?")],
        "language": "Hindi"
    },
    config=config
)
response.content

'आपने पूछा था कि मौसम कैसा है। \n\n(You asked how the weather is.)'